# Custom Objective Function

pyPESTO can not only do parameter estimation for PEtab and AMICI models, but is able to do so on any provided function.
This is done by providing the objective with the function as well as possibly gradient and hessian.
In this notebook, we will show a few different ways on how to do this. As sometimes manually providing the gradient and hessian is tedious, we will try to emphasize on the importance of those two.

After this notebook, you should ...
* ... be able to create an objective from a given function.
* ... be able to potentially add a gradient and hessian to the objective.
* ... be able to run parameter estimation on the objective.
* ... know the importance of gradient and hessian in terms of optimization speed and accuracy.

In [ ]:
# install if not done yet
# %pip install pypesto --quiet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

import pypesto
import pypesto.visualize as visualize

%matplotlib inline